In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
import numpy as np
import timeit

In [2]:
# # Import API
import requests
from config import weather_api_key
from citipy import citipy

In [3]:
# Step 1: Create a new set of 2,000 random latitudes and longitudes

# Generate random latitudes and longitudes
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
longs = np.random.uniform(low=-180.000, high=180.000, size=2000)

# Zip latitudes and longitudes together
lat_longs = zip(lats, longs)

# Create coordinates
coordinates = list(lat_longs)
len(coordinates)

2000

In [4]:
# Step 2: Get the nearest city using the citipy module.
cities = []

# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    if city not in cities:
        cities.append(city)

# City Count
len(cities)

773

In [5]:
# Step 3: Perform an API call with the OpenWeatherMap.

# URL
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [8]:
# Step 4: Retrieve the following information from the API call:

# Import the datetime module from the datetime library.
from datetime import datetime

# Create an empty list to hold the weather data.
city_data = []

# Print the beginning of the logging.
print("-----------------------------")
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in our list.
for i in range(len(cities)):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + cities[i]

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    
    # Add 1 to the record count.
    record_count += 1

# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather["weather"][0]["description"]
        # Convert the date to ISO standard.
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_description})

# If an error is experienced, skip the city.
    except KeyError:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

-----------------------------
Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 16 | rikitea
Processing Record 2 of Set 16 | butaritari
Processing Record 3 of Set 16 | atuona
Processing Record 4 of Set 16 | taolanaro
City not found. Skipping...
Processing Record 5 of Set 16 | clyde river
Processing Record 6 of Set 16 | hermanus
Processing Record 7 of Set 16 | puerto ayora
Processing Record 8 of Set 16 | mar del plata
Processing Record 9 of Set 16 | castro
Processing Record 10 of Set 16 | laguna
Processing Record 11 of Set 16 | arraial do cabo
Processing Record 12 of Set 16 | longyearbyen
Processing Record 13 of Set 16 | bluff
Processing Record 14 of Set 16 | mataura
Processing Record 15 of Set 16 | tasiilaq
Processing Record 16 of Set 16 | cherskiy
Processing Record 17 of Set 16 | daru
Processing Record 18 of Set 16 | te anau
Processing Record 19 of Set 16 | rio grande
Processing Record 20 of Set 16 | guider
Processing Record 21 of Set 16 | nikolsko

In [9]:
# Step 5: Add the data to a new DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Rikitea,PF,-23.1203,-134.9692,75.11,70,97,12.41,overcast clouds
1,Butaritari,KI,3.0707,172.7902,82.94,77,94,14.34,overcast clouds
2,Atuona,PF,-9.8000,-139.0333,79.61,64,6,8.90,clear sky
3,Clyde River,CA,70.4692,-68.5914,30.43,86,75,3.44,broken clouds
4,Hermanus,ZA,-34.4187,19.2345,54.90,76,98,6.26,overcast clouds
5,Puerto Ayora,EC,-0.7393,-90.3518,76.55,84,18,10.00,few clouds
6,Mar Del Plata,AR,-38.0023,-57.5575,52.12,55,0,1.99,clear sky
7,Castro,BR,-24.7911,-50.0119,56.23,99,100,2.51,overcast clouds
8,Laguna,US,38.4210,-121.4238,98.44,30,1,1.01,clear sky
9,Arraial Do Cabo,BR,-22.9661,-42.0278,75.16,83,75,4.61,broken clouds


In [10]:
# Step 6: Export the DataFrame as a CSV file, and 
# save it as WeatherPy_Database.csv in the Weather_Database folder.

output_data_file = "Weather_Database/WeatherPy_Database.csv"
city_data_df.to_csv(output_data_file, index_label="City_ID")